In [ ]:
import sys

from qick import *

import numpy as np
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt

In [ ]:
# Load bitstream with custom overlay
soc = QickSoc()
soccfg = soc

print(soccfg)

In [ ]:
#########################
### Weird Pulse Build ###
#########################
class PulseTest(AveragerProgram):
    def initialize(self):
        # Set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"], nqz=1)
        
        # Declare First Pulse (going up).
        env_cfg = self.cfg['up']
        length = self.us2cycles(env_cfg['length'],gen_ch=self.cfg['gen_ch'])
        N = length * self.soccfg['gens'][self.cfg['gen_ch']]['samps_per_clk']
        x = np.arange(N)/N
        y = (env_cfg['G1']-env_cfg['G0']) * x + env_cfg['G0']        
        self.add_pulse(ch=self.cfg['gen_ch'], name='up', idata=y)
        
        # Declare Second Pulse (constant).
        env_cfg = self.cfg['center']
        length = self.us2cycles(env_cfg['length'],gen_ch=self.cfg['gen_ch'])
        N = length * self.soccfg['gens'][self.cfg['gen_ch']]['samps_per_clk']
        y = self.cfg['center']['G'] * np.ones(N)
        self.add_pulse(ch=self.cfg['gen_ch'], name='center', idata=y)
        
        # Declare Third Pulse (going down).
        env_cfg = self.cfg['down']
        length = self.us2cycles(env_cfg['length'],gen_ch=self.cfg['gen_ch'])
        N = length * self.soccfg['gens'][self.cfg['gen_ch']]['samps_per_clk']
        x = np.arange(N)/N
        y = (env_cfg['G1']-env_cfg['G0']) * x + env_cfg['G0']        
        self.add_pulse(ch=self.cfg['gen_ch'], name='down', idata=y) 
        
        # First channel.
        freq = self.freq2reg(self.cfg['pulse_freq'], gen_ch=self.cfg['gen_ch'])
        self.default_pulse_registers(ch     = self.cfg['gen_ch'], 
                                     freq   = freq,
                                     phase  = 0,
                                     phrst  = 0,
                                     gain   = self.cfg['pulse_gain'],
                                     style  = 'arb',
                                     )

        # Registers for time control of central pulse.
        self.regwi(1,10,0) # Time accumulation for sync.
        self.regwi(1,11,self.us2cycles(self.cfg['center']['length']))
        
        self.synci(200)  # give processor some time to configure pulses
        
        # Trigger.
        self.trigger(pins=[0],t=40)

    def body(self):
        
        # UP Pulse.
        self.set_pulse_registers(ch=self.cfg['gen_ch'], waveform="up")
        self.pulse(ch=self.cfg['gen_ch'])
        
        # CENTER Pulse.
        self.set_pulse_registers(ch=self.cfg['gen_ch'], waveform="center", mode="periodic")
        self.pulse(ch=self.cfg['gen_ch'])
        
        # Sync to allow repeating center pulse.
        #LW = self.us2cycles((self.cfg['center']['N']-1) * self.cfg['center']['length'])
        #self.synci(LW)
        
        # Sync.
        self.sync(1,10)
        
        # DOWN Pulse.
        self.set_pulse_registers(ch=self.cfg['gen_ch'], waveform="down")
        self.pulse(ch=self.cfg['gen_ch'])                
        
        # Accumulate time.
        self.math(1,10,10,"+",11)
        
        # Period.
        self.sync_all()
        self.synci(self.us2cycles(self.cfg['gap']))
        
# Generic parameters.
config={"gen_ch"      : 4,
        "reps"        : 7,
        "pulse_gain"  : 30000, # [DAC units]
        "pulse_freq"  : 500, # [MHz]        
        "gap"         : 0.05
       }

# Envelopes.
config['up'] = {
    "length" : 0.05,
    "G0"     : 1000,
    "G1"     : 20000
}
config['center'] = {
    "length" : 0.01,
    "G"      : 6345,
    "N"      : 1,
}
config['down'] = {
    "length" : 0.1,
    "G0"     : 15000,
    "G1"     : 2000    
}

prog = PulseTest(soccfg, config)
prog.config_all(soccfg)
soc.tproc.start()        

In [ ]:
print(prog)